In [9]:
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer

os.chdir('/home/onyxia/work/synthetic-data-sdc/')
os.environ["MLFLOW_TRACKING_URI"] = "https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/"

# Données

In [13]:
variables = ['sex','age','agegr','placesize','edu','socprof','marital','ls','depress','trust','trustfam','trustneigh','sport','nofriend','smoke','alcabuse','alcsol','wkabint','englang','height','weight','bmi']

var_puf65 = ["AAC","ACTEU","AGE6","ANCCHOM","ANCEMPL4","ANCSSEMP","CHPUB","CL_EMPLOI","COUPL_LOG","DEMNE","DIP7","DISPONE","DISPPLC","ENFRED","EXTRIAN","HALOR","HEFFEMP","HEFFTOT",
           "HHABEMP","HHABTOT","METRODOM","MRANE","MRBNE","MRCNE","MRDNE","MRENE","MRFNE","MRGNE","MRHNE","MRINE","MRJNE","NAFANTG004N","NAFG004UN","NAFG010UN","NAFG017UN",
           "NAFG021UN","NOI","OFFICC","PASTRA","PASTRB","PCS1","PCS1Q","PUB3FP","RABS","RAISDISPPLC","RAISNDISPONE","RAISNRECNE","RAISNSOUNE","RAISTP","RECNE","SALTYP","SEXE",
           "SOU_C","SOUSEMPL","SOUSEMPLR","STATUT","STATUTDET","STC","STCOMM2020","STPLC","TEMP","TPPRED","TRAREF","TXTPPRED","TYPLOG5"]

fac_puf65 = ["AAC","ACTEU","AGE6","ANCCHOM","ANCEMPL4","ANCSSEMP","CHPUB","CL_EMPLOI","COUPL_LOG","DEMNE","DIP7","DISPONE","DISPPLC","ENFRED","HALOR","METRODOM","MRANE","MRBNE",
           "MRCNE","MRDNE","MRENE","MRFNE","MRGNE","MRHNE","MRINE","MRJNE","NAFANTG004N","NAFG004UN","NAFG010UN","NAFG017UN","NAFG021UN","NOI","OFFICC","PASTRA","PASTRB",
           "PCS1","PCS1Q","PUB3FP","RABS","RAISDISPPLC","RAISNDISPONE","RAISNRECNE","RAISNSOUNE","RAISTP","RECNE","SALTYP","SEXE","SOU_C","SOUSEMPL","SOUSEMPLR","STATUT",
           "STATUTDET","STC","STCOMM2020","STPLC","TEMP","TPPRED","TRAREF","TXTPPRED","TYPLOG5"]

fac_puf71 = ["AAC","ACTEU","AGE6","ANCCHOM","ANCEMPL4","ANCSSEMP","CHPUB","CL_EMPLOI","COUPL_LOG","DEMNE","DIP7","DISPONE","DISPPLC","ENFRED","HALOR","IDENT","ISCO2","METRODOM","MRANE","MRBNE","MRCNE","MRDNE","MRENE","MRFNE","MRGNE","MRHNE","MRINE","MRJNE","NAFANTG004N","NAFANTG088N","NAFG004UN","NAFG010UN","NAFG017UN",
           "NAFG021UN","NAFG038UN","NAFG088UN","NOI","OFFICC","PASTRA","PASTRB","PCS1","PCS1Q","PCS2","PUB3FP","RABS","RAISDISPPLC","RAISNDISPONE","RAISNRECNE","RAISNSOUNE","RAISTP","RECNE","SALTYP","SEXE",
           "SOU_C","SOUSEMPL","SOUSEMPLR","STATUT","STATUTDET","STC","STCOMM2020","STPLC","TEMP","TPPRED","TRAREF","TXTPPRED","TYPLOG5"]

var_puf71 = ["AAC","ACTEU","AGE6","ANCCHOM","ANCEMPL4","ANCSSEMP","CHPUB","CL_EMPLOI","COUPL_LOG","DEMNE","DIP7","DISPONE","DISPPLC","ENFRED","EXTRIAN","HALOR","HEFFEMP","HEFFTOT",
           "HHABEMP","HHABTOT","IDENT","ISCO2","METRODOM","MRANE","MRBNE","MRCNE","MRDNE","MRENE","MRFNE","MRGNE","MRHNE","MRINE","MRJNE","NAFANTG004N","NAFANTG088N","NAFG004UN","NAFG010UN","NAFG017UN",
           "NAFG021UN","NAFG038UN","NAFG088UN","NOI","OFFICC","PASTRA","PASTRB","PCS1","PCS1Q","PCS2","PUB3FP","RABS","RAISDISPPLC","RAISNDISPONE","RAISNRECNE","RAISNSOUNE","RAISTP","RECNE","SALTYP","SEXE",
           "SOU_C","SOUSEMPL","SOUSEMPLR","STATUT","STATUTDET","STC","STCOMM2020","STPLC","TEMP","TPPRED","TRAREF","TXTPPRED","TYPLOG5"]

num = ["EXTRIAN", "HEFFEMP", "HEFFTOT", "HHABEMP", "HHABTOT"]

original = pd.read_csv('SDV/df_original.csv', names = variables)
puf65 = pd.read_csv('TableEvaluator/puf.csv', names = var_puf65)

for colonne in num:
    puf65[colonne] = pd.to_numeric(puf65[colonne], errors='coerce')
    
for colonne in fac_puf65:
    puf65[colonne] = puf65[colonne].astype(str)


/tmp/ipykernel_1602/3881097872.py:25: DtypeWarning: Columns (1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  puf65 = pd.read_csv('TableEvaluator/puf.csv', names = var_puf65)


# Metadata

In [14]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(puf65)
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "AAC": {
            "sdtype": "categorical"
        },
        "ACTEU": {
            "sdtype": "categorical"
        },
        "AGE6": {
            "sdtype": "categorical"
        },
        "ANCCHOM": {
            "sdtype": "categorical"
        },
        "ANCEMPL4": {
            "sdtype": "categorical"
        },
        "ANCSSEMP": {
            "sdtype": "categorical"
        },
        "CHPUB": {
            "sdtype": "categorical"
        },
        "CL_EMPLOI": {
            "sdtype": "categorical"
        },
        "COUPL_LOG": {
            "sdtype": "categorical"
        },
        "DEMNE": {
            "sdtype": "categorical"
        },
        "DIP7": {
            "sdtype": "categorical"
        },
        "DISPONE": {
            "sdtype": "categorical"
        },
        "DISPPLC": {
            "sdtype": "categorical"
        },
        "ENFRED": {
            "sdtype": "categorical"
   

# Simulations

In [15]:
import itertools
import yaml
from yaml.loader import SafeLoader
import mlflow
import mlflow.pyfunc
import joblib

In [ ]:
with open("SDV/config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

class TVAEWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None

    def load_context(self, context):
        self.model = joblib.load(context.artifacts["model_path"])

    def predict(self, context, model_input):
        return self.model.sample(len(model_input))

remote_server_uri = os.environ["MLFLOW_TRACKING_URI"]
experiment_name = 'Default'
run_name = 'sim_puf_tvae'

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)

for i in range(20):
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param('run_name', run_name)
    
        #Entraînement
        start_time = time.time()
        tvae = TVAESynthesizer(metadata, verbose=True)
        tvae.fit(puf65)
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        mlflow.log_metric("elapsed_time", elapsed_time)
        print(f"Temps pris entraîner le modèle : {elapsed_time} secondes")

        puf65_tvae = tvae.sample(puf65.shape[0])
        puf65_tvae.to_csv(f"SDV/puf65_tvae_{i}.csv")
        
        params_tvae = tvae.get_parameters()
    
        # Log des paramètres du modèle
        for param_key, param_value in params_tvae.items():
            with mlflow.start_run(nested=True):
                mlflow.log_param(param_key, param_value)
    
        loss = tvae.get_loss_values()
        losses = loss['Loss'].tolist()
        for epoch, loss in enumerate(losses):
            mlflow.log_metric("loss", loss, step=epoch)
    
        model_path = "tvae_model.pkl"
        joblib.dump(tvae, model_path)
        mlflow.pyfunc.log_model(
            artifact_path="tvae_model",
            python_model=TVAEWrapper(),
            artifacts={"model_path": model_path},
        )
    mlflow.end_run()

Loss: -4.166: 100%|██████████| 300/300 [1:42:59<00:00, 20.60s/it]


Temps pris entraîner le modèle : 6276.590941905975 secondes


2024/08/25 20:22:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run silent-shark-851 at: https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/#/experiments/0/runs/d85143a689314a4fa8e5b8c2be74f0df.
2024/08/25 20:22:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/#/experiments/0.
2024/08/25 20:22:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run skillful-tern-445 at: https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/#/experiments/0/runs/6bc7d27a607643e79a32f57bcb8c24c8.
2024/08/25 20:22:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/#/experiments/0.
2024/08/25 20:22:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run auspicious-wasp-533 at: https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/#/experiments/0/runs/023a5dd3d1a14ca4b652195b8af1b394.
20